In [ ]:
import numpy as np
import os
import glob

In [ ]:
t_start = 16650
t_end = 17000
dt = 0.1
seq_dir = "02/"
save_dir = "02_processed/"

voxel_resolution = 0.2
grid_lengths = np.array([40.0, 40.0, 4.0])
NUM_CLASSES = 25

grid_dim = 2 * grid_lengths / voxel_resolution
grid_dim = [int(grid_dim[0]), int(grid_dim[1]), int(grid_dim[2]), NUM_CLASSES] # 0 is free
print(grid_dim)

x = np.arange(-grid_lengths[0], grid_lengths[0], voxel_resolution) + voxel_resolution/2
y = np.arange(-grid_lengths[1], grid_lengths[1], voxel_resolution) + voxel_resolution/2
z = np.arange(-grid_lengths[2], grid_lengths[2], voxel_resolution) + voxel_resolution/2
xv, yv, zv = np.meshgrid(x, y, z)

In [ ]:
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

sensors = glob.glob(seq_dir + "velodyne*")
sensors = sorted([int(sensor.split("velodyne")[1]) for sensor in sensors])
ego_sensor = sensors[1]

In [ ]:
times = []

t_from = open(seq_dir + "times0.txt", "r")
for t_stamp in t_from.readlines():
    t_frame = t_stamp.split(", ")[0]
    if int(t_frame) < t_start:
        continue
    if int(t_frame) >= t_end:
        continue
    t_frame = (float(t_frame) - t_start) * dt
    times.append(t_frame)
t_from.close()

times = np.array(times)
print(times)
np.savetxt(save_dir + 'times.txt', times)

In [ ]:
def get_frame_str(t_frame, t_start, t_end):
    if int(t_frame) < t_start:
        return None
    if int(t_frame) >= t_end:
        return None
    t_frame = int(t_frame) - t_start
    return str(t_frame).zfill(6)

In [ ]:
# Original view
poses = {}
sorted_poses_all = {}
inv_first = None 
for sensor in sensors:
    poses[sensor] = {}
    # Get poses
    for pose_file in os.listdir(seq_dir + "pose" + str(sensor)):
        t_frame = pose_file.split(".")[0]
        frame_str = get_frame_str(t_frame, t_start, t_end)
        if frame_str:
            pose = np.load(seq_dir + "pose" + str(sensor) + "/" + pose_file)
            poses[sensor][frame_str] = pose

    # Sort poses
    sorted_poses = [poses[sensor][fr] for fr in sorted(poses[sensor].keys())]
    # Make first pose origin
    if sensor == 0:
        inv_first = np.linalg.inv(sorted_poses[0])
    for i in range(len(sorted_poses)):
        sorted_poses[i] = np.matmul(inv_first, sorted_poses[i])
        sorted_poses[i] = sorted_poses[i].reshape(-1)[:12]

    sorted_poses = np.array(sorted_poses)
    if sensor == 0:
        np.savetxt(save_dir + '/poses.txt', sorted_poses)
    sorted_poses_all[sensor] = sorted_poses

In [ ]:
# Labels for ego sensor
def save_labels(view_num, query):
    to_folder = save_dir + "labels"
    if query:
        to_folder += "_query"
    to_folder += "/"
    
    if not os.path.exists(to_folder):
        os.mkdir(to_folder)
    
    for label_file in os.listdir(seq_dir + "labels" + str(view_num)):
        t_frame = label_file.split(".")[0]
        frame_str = get_frame_str(t_frame, t_start, t_end)
        if frame_str:
            label = np.load(seq_dir + "labels" + str(view_num) + "/" + label_file)
            label.astype('uint32').tofile(to_folder + frame_str + ".label")
            values, counts = np.unique(label, return_counts=True)
            print(values, counts)
        
save_labels(0, False)

In [ ]:
# Labels
def save_points(view_num, query):
    to_folder_points = save_dir + "velodyne"
    to_folder_flow = save_dir + "predictions"
    if query:
        to_folder_points += "_query"
        to_folder_flow += "_query"
    to_folder_points += "/"
    to_folder_flow += "/"
    
    if not os.path.exists(to_folder_points):
        os.mkdir(to_folder_points)
    if not os.path.exists(to_folder_flow):
        os.mkdir(to_folder_flow)
    
    for point_file in os.listdir(seq_dir + "velodyne" + str(view_num)):
        t_frame = point_file.split(".")[0]
        frame_str = get_frame_str(t_frame, t_start, t_end)
        if frame_str:
            points = np.load(seq_dir + "velodyne" + str(view_num) + "/" + point_file) # Point cloud
            instances = np.load(seq_dir + "instances" + str(view_num) + "/" + point_file) # Instances per point
            velocities = np.load(seq_dir + "velocities" + str(view_num) + "/" + point_file) # Per-instance velocity
            flow = np.zeros(points.shape, dtype=np.float32) # Actual movement of things
            for row in velocities:
                ind = int(row[0])
                velocity = row[1:]
                flow[instances == ind] = velocity
 
            points = np.c_[points, np.zeros(points.shape[0])] # Dummy intensity
            points.astype('float32').tofile(to_folder_points + frame_str + ".bin")
            flow.astype('float32').tofile(to_folder_flow + frame_str + ".bin")
        
save_points(0, False)

In [ ]:
# Helper to create voxel grid
def get_info(sensor, frame_str, seq_dir):
    points = np.load(seq_dir + "velodyne" + str(sensor) + "/" + frame_str + ".npy") # Point cloud
    instances = np.load(seq_dir + "instances" + str(sensor) + "/" + frame_str + ".npy") # Instances per point
    velocities = np.load(seq_dir + "velocities" + str(sensor) + "/" + frame_str + ".npy") # Per-instance velocity
    flow = np.zeros(points.shape, dtype=np.float32) # Actual movement of things
    for row in velocities:
        ind = int(row[0])
        velocity = row[1:]
        flow[instances == ind] = velocity
    label = np.load(seq_dir + "labels" + str(sensor) + "/" + frame_str + ".npy")
    return [points, instances, flow, label]

In [ ]:
# Initial transforms (other lidar to ego sensor)
inv_transforms = {} 
for pose_file in os.listdir(seq_dir + "pose0"):
    t_frame = pose_file.split(".")[0]
    frame_str = get_frame_str(t_frame, t_start, t_end)
    if frame_str:
        ego_pose = np.load(seq_dir + "pose0/" + pose_file)
        for sensor in sensors:
            sensor_pose = np.load(seq_dir + "pose" + str(sensor) + "/" + pose_file)
            inv_sensor = np.linalg.inv(sensor_pose)
            inv_transforms[sensor] = np.matmul(ego_pose, inv_sensor)

In [ ]:
def add_point(point, label, grid):
    voxel = np.floor((point + grid_lengths) / voxel_resolution)
    coords = [int(voxel[0]), int(voxel[1]), int(voxel[2]), int(label)]
    if np.all(np.array(coords) < np.array(grid_dim)):
        grid[coords[0], coords[1], coords[2], coords[3]] += 1
    return grid

# Vectorized
def add_points(points, labels, grid):
    # All voxels
    voxels = np.floor((points + grid_lengths) / voxel_resolution).astype(int)
    voxels = np.hstack((voxels, np.reshape(labels, (-1, 1))))
    # Voxels within grid (check max and min)
    valid_voxels = np.all(voxels < np.reshape(np.array(grid_dim), (1, -1)), axis=1) # Max
    valid_voxels = voxels[valid_voxels]
    valid_inds = np.all(valid_voxels >= 0, axis=1) # Min
    valid_voxels = valid_voxels[valid_inds]
    # Convert to list
    valid_coords = [list(valid_voxels[:, 0]), list(valid_voxels[:, 1]), list(valid_voxels[:, 2]), list(valid_voxels[:, 3])]
    
    grid[valid_coords] += 1
    return grid

In [ ]:
# Add points along ray to grid
def ray_trace(point, label, grid, sample_spacing):
    vec_norm = np.linalg.norm(point)
    vec_angle = point / vec_norm
    dists = np.arange(vec_norm, 0, -sample_spacing)
    new_points = np.reshape(dists, (-1, 1)) * np.reshape(vec_angle, (1, 3))
    labels = [0] * new_points.shape[0]
    # End Point is label, free points all 0
    labels[0] = label
    grid = add_points(new_points, labels, grid)
    return grid

In [ ]:
# Whether any measurements were found
if not os.path.exists(save_dir + "evaluation"):
    os.mkdir(save_dir + "evaluation")
    

# Create voxel grid
for pose_file in os.listdir(seq_dir + "pose0"):
    t_frame = pose_file.split(".")[0]
    frame_str = get_frame_str(t_frame, t_start, t_end)
    voxel_grid = np.zeros((grid_dim))
    if frame_str:
        for sensor in sensors:
            [points, instances, flow, label] = get_info(sensor, t_frame, seq_dir) # Get info for sensor at frame
            transformed_points = np.matmul(points, inv_transforms[sensor][:3, :3]) # Convert points to ego frame
            transformed_points = transformed_points + inv_transforms[sensor][3, :3]
            for i in range(transformed_points.shape[0]):
                voxel_grid = ray_trace(transformed_points[i, :], label[i], voxel_grid, voxel_resolution)
        # Save
        valid_cells = np.sum(voxel_grid, axis=3) > 0
        labels = np.argmax(voxel_grid, axis=3)
        values, counts = np.unique(labels[valid_cells], return_counts=True)
        print(frame_str, values, counts)
        valid_x = xv[valid_cells]
        valid_y = yv[valid_cells]
        valid_z = zv[valid_cells]
        valid_points = np.stack((valid_x, valid_y, valid_z)).T
        valid_labels = labels[valid_cells]
        valid_points.astype('float32').tofile(save_dir + "/evaluation/" + frame_str + ".bin")
        valid_labels.astype('uint32').tofile(save_dir + "/evaluation/" + frame_str + ".label")

In [ ]:
LABEL_COLORS = np.array([
    (255, 255, 255), # None (free) 0
    (70, 70, 70),    # Building 1
    (100, 40, 40),   # Fences 2
    (55, 90, 80),    # Other 3
    (220, 20, 60),   # Pedestrian 4
    (153, 153, 153), # Pole 5
    (157, 234, 50),  # RoadLines 6
    (128, 64, 128),  # Road 7
    (244, 35, 232),  # Sidewalk 8
    (107, 142, 35),  # Vegetation 9
    (0, 0, 142),     # Vehicle 10
    (102, 102, 156), # Wall 11
    (220, 220, 0),   # TrafficSign 12
    (70, 130, 180),  # Sky 13
    (81, 0, 81),     # Ground 14
    (150, 100, 100), # Bridge 15
    (230, 150, 140), # RailTrack 16
    (180, 165, 180), # GuardRail 17
    (250, 170, 30),  # TrafficLight 18
    (110, 190, 160), # Static 19
    (170, 120, 50),  # Dynamic 20
    (45, 60, 150),   # Water 21
    (145, 170, 100), # Terrain 22
]) / 255.0 # normalize each channel [0-1] since is what Open3D uses